In [1]:
import datetime
import os
from mdagent import MDAgent

In [2]:
prompt7 = "Simulate 1ZNI for 1 ns at a temperature of 300 K."
llm_var = "accounts/fireworks/models/llama-v3p1-70b-instruct"
tools = "all"
agent = MDAgent(agent_type="Structured", model=llm_var, top_k_tools=tools)
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date and time:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)
print("LLM: ",agent.llm.model_name,"\nTemperature: ",agent.llm.temperature)

date and time: 2024-09-12
time: 13:59:37
LLM:  accounts/fireworks/models/llama-v3p1-70b-instruct 
Temperature:  0.1


In [3]:
output = agent.run(prompt7)

Thought: To simulate 1ZNI for 1 ns at a temperature of 300 K, I need to set up and run a molecular dynamics simulation. I will use the SetUpandRunFunction tool to achieve this.

Action:
```
{
  "action": "SetUpandRunFunction",
  "action_input": {
    "pdb_id": "1ZNI",
    "forcefield_files": ["amber99sbildn.xml", "tip3p.xml"],
    "save": true,
    "system_params": {
      "nonbondedMethod": "PME",
      "nonbondedCutoff": "1.0 * nanometers",
      "ewaldErrorTolerance": 0.00001,
      "constraints": "HBonds",
      "rigidWater": true,
      "constraintTolerance": 0.00001,
      "solvate": true
    },
    "integrator_params": {
      "integrator_type": "LangevinMiddle",
      "Temperature": "300 * kelvin",
      "Friction": "1.0 / picoseconds",
      "Timestep": "0.002 * picoseconds",
      "Pressure": "1.0 * bar"
    },
    "simulation_params": {
      "Ensemble": "NVT",
      "Number of Steps": 500000,
      "record_interval_steps": 100,
      "record_params": ["step", "potentialEner

In [4]:
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date and time:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)

date and time: 2024-09-12
time: 21:12:28


In [5]:
registry = agent.path_registry
print("ckpt_dir:",os.path.basename(registry.ckpt_dir))
paths_and_descriptions = registry.list_path_names_and_descriptions()
print("\n".join(paths_and_descriptions.split(",")))

ckpt_dir: ckpt_7
Files found in registry: 1ZNI_135944: PDB file downloaded from RSCB
 PDBFile ID: 1ZNI_135944
 1ZNI_135950: Cleaned File:  Removed Heterogens
 and Water Removed.  Replaced Nonstandard Residues. Added Hydrogens at pH 7.0. Missing Atoms Added and replaces nonstandard residues. 
 top_sim0_135954: Initial positions for simulation sim0_135954
 1ZNI_144111: Cleaned File:  Removed Heterogens
 and Water Removed.  Replaced Nonstandard Residues. Added Hydrogens at pH 7.0. Missing Atoms Added and replaces nonstandard residues. 
 top_sim0_144115: Initial positions for simulation sim0_144115
 sim0_144115: Basic Simulation of Protein 1ZNI_144111
 rec0_144119: Simulation trajectory for protein 1ZNI_144111 and simulation sim0_144115
 rec1_144119: Simulation state log for protein 1ZNI_144111 and simulation sim0_144115
 rec2_144119: Simulation pdb frames for protein 1ZNI_144111 and simulation sim0_144115


In [6]:
import re
import os
match = re.search(r"rec0_\d+", paths_and_descriptions)
traj_id = match.group(0)
traj_path = registry.get_mapped_path(traj_id)
match = re.search(r"top_sim0_\d+", paths_and_descriptions)
top_id = match.group(0)
top_path = registry.get_mapped_path(top_id)
assert os.path.exists(traj_path), "Trajectory file doesn't exist"
assert os.path.exists(top_path), "Topology file doesn't exist"
assert traj_path != top_path
assert traj_path.endswith(".dcd")
assert top_path.endswith(".pdb")
print(f"It's asserted that file paths for {traj_id} and {top_id} exist")

It's asserted that file paths for rec0_144119 and top_sim0_135954 exist


In [7]:
import mdtraj as md
traj = md.load(traj_path, top=top_path)
print(f"Simulation with {traj.n_atoms} atoms for {traj.n_frames} frames.")

Simulation with 19766 atoms for 5010 frames.


# Experiment Result: ✅❌

### Completed without Exception or TimeOut Errors ✅
### Attempted all necessary steps ✅
### Completed without Hallucination ✅
### Logic make sense ✅
### Correct Answer ✅